## Spots that are always excluded:
##### Test spots
##### Spots marked as problematic
##### Suscpicious for analysis == 'Yes' spots
##### Fake Sale spots: This means that Reason for Cancelling Survey is any combination which contains 'Fake Sale'
##### Spots whose stop date was before 2018


## CAN CANCEL:
##### Spots which cancelled more than 2 months before their Available Cancellation Date are excluded
##### Spots whose Available Cancellation Date is 2 or more months after the date of analysis
##### Months during which spots couldn't have cancelled according to the above conditions

## wo CB sets:
##### Closed Business, Sold Business and Non-payment spots are excluded. This means Reason for Cancelling Survey is any combination which contains 'Closed Business', 'Closed/Sold Business', 'Sold Business', 'Sold/Closed Business' or 'Non-payment'
## canc conf event
##### Spots for which Date Cancellation Confirmed is more than 60 days after Date Cancellation Requested

#### ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import pandas as pd
import numpy as np
import importlib
import get_stopped_doing_something_variables
import prepare_for_the_models
import fit_tv_cox_models
#import read_vars_for_separate_models_for_dependent_vars
import read_a_combination_of_variables
import yaml
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display, Markdown

In [2]:
# read the yaml file with a list of parameters needed for the report
with open(r'./parameters/stopped_doing_something_report_parameters.yaml') as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)

date_of_analysis = parameters['date_of_analysis']
date_dir = date_of_analysis.replace('-', '_')
### name of the data directory ###
churn_based_on_behaviour_dir = parameters['churn_based_on_behaviour_dir']
### penalizer value when fitting the models ###
penalizer = parameters['penalizer']
### model type ###
model_type = parameters['model_type']
### coefficient and p values when dropping unsignificant variables ###
coefficient_limit_for_numerical_vars = parameters['coefficient_limit_for_numerical_vars']
coefficient_limit_for_cat_vars = parameters['coefficient_limit_for_cat_vars']
p_limit = parameters['p_limit']
additional_higher_p_limit = parameters['additional_higher_p_limit']
additional_lower_p_limit = parameters['additional_lower_p_limit']

# read the yaml file with data set parameters #
# data_set_name = input('Data set name: ')
data_set_name = 'CAN_CANCEL_spots_with_CB_cancellation_confirmed'
with open(r'./parameters/data_sets.yaml') as file:
    data_sets_parameters = yaml.load(file, Loader=yaml.FullLoader)
    
spots_set = data_sets_parameters[data_set_name]['spots_set']
with_wo_CB = data_sets_parameters[data_set_name]['with_wo_CB']
event_date_full_name = data_sets_parameters[data_set_name]['event_date_type']

if with_wo_CB == 'with_CB':
    with_wo_CB_boolean = True
else:
    with_wo_CB_boolean = False

if event_date_full_name == 'cancellation_requested':
    event_date = 'canc_req'
elif event_date_full_name == 'cancellation_confirmed':
    event_date = 'canc_conf'

In [3]:
##### base columns ##### 
base_cols = ['spot_id',\
            'time',\
            'event']

In [4]:
##### Model 0: all variables - prepare data for the model ####
cols_to_use = read_a_combination_of_variables.\
main(model_number=0, dir_name='combinations_of_variables_that_are_not_dependent/')
#### get behavioural variables ####
(variables_to_use_for_the_model, did_something_before_vars, did_something_last_month_vars) = \
get_stopped_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
cols = base_cols + variables_to_use_for_the_model + \
did_something_before_vars + did_something_last_month_vars


#### get data for the model ###
(data, base_df, df_timeline_all_vars) = \
prepare_for_the_models.get_data_for_the_MV_Cox_model(date_of_analysis=date_of_analysis, spots_set=spots_set, \
                            with_wo_CB=with_wo_CB, event_date=event_date, columns=cols, data_dir=churn_based_on_behaviour_dir,\
                            C = 100)
    
model_numbers = \
read_a_combination_of_variables.get_a_list_of_model_numbers(dir_name='combinations_of_variables_that_are_not_dependent/')

In [5]:
for model_number in model_numbers[1:]:
    cols_to_use = read_a_combination_of_variables.\
    main(model_number=model_number, dir_name='combinations_of_variables_that_are_not_dependent/')
    
    model_name = \
    read_a_combination_of_variables.get_model_names(model_number=model_number, \
                                                    dir_name='combinations_of_variables_that_are_not_dependent/')
    display(Markdown("# Model "+ str(model_number) + ": " + model_name))

    #### get behavioural variables ####
    (variables_to_use_for_the_model, did_something_before_vars, did_something_last_month_vars) = \
    get_stopped_doing_something_variables.main(date_of_analysis=date_of_analysis, variables_to_use_for_the_model=cols_to_use)
    cols = variables_to_use_for_the_model + \
    did_something_before_vars + did_something_last_month_vars

    #### data for the model ###
    df_timeline = df_timeline_all_vars.copy()
    vars_that_stay = ['spot_id', 'start', 'stop', 'event']+\
    [x for x in cols if x not in base_cols]+\
    [x for x in df_timeline.columns if 'spot_category_' in x or 'metro_area_' in x]
    df_timeline.drop([x for x in df_timeline.columns if x not in vars_that_stay], axis = 1, inplace = True)

    ### variables to skip ###
    df_timeline.isnull().sum().sum() #OK
    skip_vars = list((df_timeline!=0).sum()[(df_timeline!=0).sum()==0].index)
    skip_vars

    ctv = fit_tv_cox_models.fit_the_models_and_print_summaries(df_timeline=df_timeline, base_df=base_df, \
                                                         date_of_analysis=date_of_analysis, model_type=model_type, \
                                                         variables_to_use_for_the_model=cols_to_use,\
                                                         coefficient_limit_for_numerical_vars=coefficient_limit_for_numerical_vars, \
                                                         coefficient_limit_for_cat_vars=coefficient_limit_for_cat_vars, p_limit=p_limit, \
                                                         additional_higher_p_limit=additional_higher_p_limit, \
                                                         additional_lower_p_limit=additional_lower_p_limit,\
                                                         skip_vars=skip_vars, penalizer=penalizer)

    ### save coefs and p values for p < 0.2 ###
    fit_tv_cox_models.save_results(df = ctv.summary.reset_index(), date_of_analysis = date_of_analysis, \
                                   data_dir = churn_based_on_behaviour_dir, dir_name='exports/coefficients_and_pvalues',\
                 results_name = 'coef_and_pvalues', spots_set = spots_set,\
                 with_wo_CB = with_wo_CB, event_date_type = event_date_full_name, p_limit=0.2, model_number=model_number,\
                                  model_type = model_type)

# Model 1: posts_on_facebook

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 18092 periods, 1506 subjects, 283 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 1506
 number of periods = 18092
  number of events = 283
partial log-likelihood = -1680.88
  time fit was run = 2021-12-21 10:00:06 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Annual Prepay                                        0.28       1.32       0.15            -0.01             0.57                 0.99                 1.78
Monthly                                              0.47       1.60       0.27            -0.06             1.00                 0.95                 2.72
Website.Views.last.month.total.log2                 -0.25       0.78       0.04            -0.34            -0.17                 0.71                 0.85
had_added_events_manually_edited_events_before       0.24       1.27       0.15            -0.05             0.53                 0.95                 1.70
had_added_events_manually_edited_events_last_3_...  -0.45       0.64       0.26            -0.96             0.07                 0.38                 1.07
had_clicked_emails_before                            0.35       1.42       0.17             0.02             0.69                 1.02                 2.00
had_clicked_emails_last_3_months                    -0.56       0.57       0.17            -0.88            -0.23                 0.41                 0.79
had_emails_sent_manually_scheduled_emails_sent_...  -0.09       0.91       0.15            -0.38             0.19                 0.69                 1.21
had_emails_sent_manually_scheduled_emails_sent_...  -0.32       0.73       0.18            -0.66             0.03                 0.52                 1.03
had_posts_on_facebook_before                         0.61       1.84       0.16             0.30             0.93                 1.34                 2.52
had_posts_on_facebook_last_month                    -0.49       0.61       0.15            -0.78            -0.21                 0.46                 0.81
had_qr_code_menu_scans_before                       -0.36       0.70       0.25            -0.84             0.13                 0.43                 1.14
had_qr_code_menu_scans_last_month                   -0.06       0.94       0.28            -0.60             0.49                 0.55                 1.62
had_reservations_submissions_before                  0.32       1.38       0.20            -0.08             0.72                 0.92                 2.05
had_reservations_submissions_last_month             -0.82       0.44       0.34            -1.49            -0.15                 0.23                 0.86
had_tickets_before                                  -0.52       0.59       0.15            -0.83            -0.22                 0.44                 0.80
had_tickets_last_3_months                            0.15       1.17       0.16            -0.15             0.46                 0.86                 1.59
had_visited_admin_before                             0.58       1.78       0.33            -0.07             1.23                 0.93                 3.41
had_visited_admin_last_month                        -0.22       0.80       0.15            -0.51             0.07                 0.60                 1.07
had_visited_qrcode_flyers_page_before               -0.59       0.55       0.26            -1.10            -0.09                 0.33                 0.91
had_visited_qrcode_flyers_page_last_3_months         0.10       1.10       0.36            -0.61             0.80                 0.54                 2.24
had_visited_special_page_before                     -0.07       0.93       0.17            -0.41           

# Model 2: changed_picture_or_text

...

<lifelines.CoxTimeVaryingFitter: fitted with 18092 periods, 1506 subjects, 283 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 1506
 number of periods = 18092
  number of events = 283
partial log-likelihood = -1677.36
  time fit was run = 2021-12-21 11:05:00 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Annual Prepay                                        0.30       1.35       0.15             0.01             0.60                 1.01                 1.82
Instagram.Add.on                                     0.21       1.23       0.12            -0.03             0.45                 0.97                 1.56
Monthly                                              0.55       1.73       0.27             0.02             1.08                 1.02                 2.94
New.email.subscriber.signups.last.month.div10       -0.08       0.93       0.06            -0.19             0.03                 0.83                 1.03
Website.Views.last.month.total.log2                 -0.24       0.78       0.05            -0.33            -0.15                 0.72                 0.86
had_added_events_manually_edited_events_before       0.24       1.27       0.15            -0.06             0.53                 0.95                 1.70
had_added_events_manually_edited_events_last_3_...  -0.41       0.66       0.26            -0.93             0.10                 0.39                 1.10
had_clicked_emails_before                            0.34       1.40       0.18            -0.01             0.68                 0.99                 1.97
had_clicked_emails_last_3_months                    -0.53       0.59       0.17            -0.86            -0.20                 0.42                 0.82
had_emails_sent_manually_scheduled_emails_sent_...  -0.13       0.87       0.15            -0.42             0.15                 0.66                 1.17
had_emails_sent_manually_scheduled_emails_sent_...  -0.29       0.75       0.18            -0.64             0.06                 0.53                 1.06
had_posts_liked_before                               0.08       1.08       0.15            -0.21             0.36                 0.81                 1.44
had_posts_liked_last_3_months                       -0.51       0.60       0.22            -0.95            -0.08                 0.39                 0.92
had_posts_on_facebook_before                         0.49       1.63       0.18             0.14             0.84                 1.15                 2.31
had_posts_on_facebook_last_month                    -0.46       0.63       0.15            -0.75            -0.17                 0.47                 0.85
had_preview_page_views_before                        0.48       1.61       0.24             0.01             0.94                 1.02                 2.55
had_preview_page_views_last_3_months                -0.05       0.95       0.15            -0.35             0.25                 0.70                 1.28
had_qr_code_menu_scans_before                       -0.36       0.69       0.25            -0.85             0.12                 0.43                 1.13
had_qr_code_menu_scans_last_month                   -0.08       0.93       0.28            -0.62             0.47                 0.54                 1.60
had_reservations_submissions_before                  0.24       1.28       0.20            -0.16             0.65                 0.85                 1.91
had_reservations_submissions_last_month             -0.83       0.44       0.34            -1.50            -0.16                 0.22                 0.86
had_tickets_before                                  -0.51       0.60       0.16            -0.82           

# Model 33: New.email.subscribers.allinclusive.last.month.div10

## without variables with p >= 0.2

### sorted by variable name

<lifelines.CoxTimeVaryingFitter: fitted with 18092 periods, 1506 subjects, 283 events>
         event col = 'event'
         penalizer = 0.001
number of subjects = 1506
 number of periods = 18092
  number of events = 283
partial log-likelihood = -1678.39
  time fit was run = 2021-12-21 11:07:12 UTC

---
                                                     coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                  
Annual Prepay                                        0.30       1.35       0.15             0.01             0.60                 1.01                 1.82
Instagram.Add.on                                     0.21       1.23       0.12            -0.03             0.45                 0.97                 1.56
Monthly                                              0.55       1.73       0.27             0.02             1.08                 1.02                 2.94
Website.Views.last.month.total.log2                 -0.27       0.77       0.04            -0.35            -0.18                 0.70                 0.83
had_added_events_manually_edited_events_before       0.24       1.27       0.15            -0.05             0.53                 0.95                 1.71
had_added_events_manually_edited_events_last_3_...  -0.43       0.65       0.26            -0.94             0.08                 0.39                 1.09
had_clicked_emails_before                            0.34       1.40       0.18            -0.01             0.68                 0.99                 1.97
had_clicked_emails_last_3_months                    -0.53       0.59       0.17            -0.85            -0.20                 0.43                 0.82
had_emails_sent_manually_scheduled_emails_sent_...  -0.14       0.87       0.15            -0.43             0.15                 0.65                 1.16
had_emails_sent_manually_scheduled_emails_sent_...  -0.28       0.75       0.18            -0.63             0.07                 0.53                 1.07
had_posts_liked_before                               0.08       1.08       0.15            -0.21             0.37                 0.81                 1.44
had_posts_liked_last_3_months                       -0.51       0.60       0.22            -0.94            -0.08                 0.39                 0.93
had_posts_on_facebook_before                         0.49       1.63       0.18             0.14             0.84                 1.15                 2.31
had_posts_on_facebook_last_month                    -0.45       0.64       0.15            -0.74            -0.16                 0.48                 0.85
had_preview_page_views_before                        0.48       1.61       0.24             0.01             0.94                 1.01                 2.55
had_preview_page_views_last_3_months                -0.05       0.95       0.15            -0.35             0.25                 0.70                 1.28
had_qr_code_menu_scans_before                       -0.37       0.69       0.25            -0.86             0.11                 0.42                 1.12
had_qr_code_menu_scans_last_month                   -0.09       0.91       0.28            -0.64             0.45                 0.53                 1.57
had_reservations_submissions_before                  0.25       1.29       0.20            -0.15             0.65                 0.86                 1.92
had_reservations_submissions_last_month             -0.82       0.44       0.34            -1.49            -0.14                 0.23                 0.87
had_tickets_before                                  -0.52       0.60       0.16            -0.82            -0.21                 0.44                 0.81
had_tickets_last_3_months                            0.15       1.17       0.16            -0.15           